# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 자모 단위 FASTTEXT

In [ ]:
!pip install hgtk

In [ ]:
!pip install konlpy

from konlpy.tag import Mecab
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 4.14 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-30 08:21:29--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.

In [ ]:
import hgtk
from tqdm import tqdm
from konlpy.tag import Mecab

def word_to_jamo(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''

    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    # 단 어 토 큰 의 자 모 단 위 분 리 결 과 를 추 가
    return decomposed_token


mecab = Mecab()

def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("final.csv")

train_data, test_data = train_test_split(df, test_size=0.20, random_state=2)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=2)

In [ ]:
df['title'][0]

'국내산 100% 전라도 배추 김치 포기 김장김치 주문'

In [ ]:
word_to_jamo(df['title'][0])

'ㄱㅜㄱㄴㅐ-ㅅㅏㄴ 100% ㅈㅓㄴㄹㅏ-ㄷㅗ- ㅂㅐ-ㅊㅜ- ㄱㅣㅁㅊㅣ- ㅍㅗ-ㄱㅣ- ㄱㅣㅁㅈㅏㅇㄱㅣㅁㅊㅣ- ㅈㅜ-ㅁㅜㄴ'

In [ ]:
print(tokenize_by_jamo(df['title'][0]))

['ㄱㅜㄱㄴㅐ-ㅅㅏㄴ', '100', '%', 'ㅈㅓㄴㄹㅏ-ㄷㅗ-', 'ㅂㅐ-ㅊㅜ-', 'ㄱㅣㅁㅊㅣ-', 'ㅍㅗ-ㄱㅣ-', 'ㄱㅣㅁㅈㅏㅇ', 'ㄱㅣㅁㅊㅣ-', 'ㅈㅜ-ㅁㅜㄴ']


In [ ]:
def tokenized(data):
  tokenized_data=[]

  for sample in tqdm(data['title'].to_list()):
      tokenzied_sample = tokenize_by_jamo(sample) # 자 소 단 위 토 큰 화
      tokenized_data.append(tokenzied_sample)

  return tokenized_data

In [ ]:
print(tokenized_data[0])

['ㄱㅜㄱㄴㅐ-ㅅㅏㄴ', '100', '%', 'ㅈㅓㄴㄹㅏ-ㄷㅗ-', 'ㅂㅐ-ㅊㅜ-', 'ㄱㅣㅁㅊㅣ-', 'ㅍㅗ-ㄱㅣ-', 'ㄱㅣㅁㅈㅏㅇ', 'ㄱㅣㅁㅊㅣ-', 'ㅈㅜ-ㅁㅜㄴ']


In [ ]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

# 1. 초 기 입 력
# jamo_sequence = ' ﾤ ￂ ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '

    while index < len(jamo_sequence):
    # 문 자 가 한 글 ( 정 상 적 인 자 모 ) 이 아 닐 경 우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

    # 문 자 가 정 상 적 인 자 모 라 면 초 성 , 중 성 , 종 성 을 하 나 의 토 큰 으 로 간 주 .
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

# 2. 자 모 단 위 토 큰 화 완 료
# tokenized_jamo : [' ﾤ ￂ ﾱ ', ' ﾧ ￌ ﾷ ', ' ﾵ ￃ ﾷ ']

        word = ''
        try:
            for jamo in tokenized_jamo:
            # 초 성 , 중 성 , 종 성 의 묶 음 으 로 추 정 되 는 경 우
                if len(jamo) == 3:
                    if jamo[2] == "-":
                    # 종 성 이 존 재 하 지 않 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1])
                    else:
                # 종 성 이 존 재 하 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
                # 한 글 이 아 닌 경 우
                else:
                    word = word + jamo

            # 복 원 중 (hgtk.letter.compose) 에 러 발 생 시 초 기 입 력 리 턴 .
            # 복 원 이 불 가 능 한 경 우 예 시 ) ' ﾤ ! ﾱ ﾧ ￌ ﾷ ﾵ ￃ ﾷ '
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                return jamo_sequence

        # 3. 단 어 로 복 원 완 료
        # word : ' 남 동 생 '

    return word

In [ ]:
jamo_to_word(tokenized_data[0][0])

'국내산'

# 학습데이터 구축
학습데이터를 만듭니다.

In [ ]:
train_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [ ]:
train_token = tokenized(train_data)
val_token = tokenized(val_data)
test_token = tokenized(test_data)

100%|██████████| 53272/53272 [00:11<00:00, 4456.03it/s]


In [ ]:
with open('train_data.txt', 'w') as out:
    for i in tqdm(range(len(train_token)), unit=' line'):
          out.write("__label__" + train_data['category'][i] + "\t" + ' '.join(train_token[i]) + '\n')

with open('val_data.txt', 'w') as out:
    for i in tqdm(range(len(val_token)), unit=' line'):
        out.write("__label__" + val_data['category'][i] + "\t" + ' '.join(val_token[i]) + '\n')

with open('test_data.txt', 'w') as out:
    for i in tqdm(range(len(test_token)), unit=' line'):
        out.write("__label__" + test_data['category'][i] + "\t" + ' '.join(test_token[i]) + '\n')

100%|██████████| 53272/53272 [00:00<00:00, 122323.12 line/s]


# 학습


In [ ]:
import fasttext

model = fasttext.train_supervised(input='train_data.txt', autotuneValidationFile='val_data.txt')

In [ ]:
model.predict(word_to_jamo("찜닭"))

(('__label__과자/떡/베이커리',), array([0.52032429]))

찜닭을 "과자/떡/베이커리" 로 구분하는 대단한 수준 

# 테스트
결과는 (샘플, 정확도, 재현율)

In [ ]:
model.test("test_data.txt")

(53272, 0.8496959002853282, 0.8496959002853282)